In [1]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import pandas as pd
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx
import onnxruntime as ort
from sklearn.metrics import accuracy_score


# Предподготовка данных

In [2]:

data = pd.read_csv('NSTU_Students_data.csv')
data = data.drop('Курс обучения', axis=1)


In [3]:
binary_fields = {
    'Пол': {'Мужской': True, 'Женский': False},
    'Вы поступили в университет, потому что сами этого хотели?': {'Да': True, 'Нет': False},
    'Вам важно получать хорошие отметки?': {'Да': True, 'Нет': False}
}

for field, mapping in binary_fields.items():
    data[field] = data[field].map(mapping)

In [4]:
fields = {
    'Сумма баллов ЕГЭ за 3 предмета': {'120-150': 1, '151-200': 2, '201-230':3, '231-260': 4, '261-300': 5 }
}
for field, mapping in fields.items():
    data[field] = data[field].map(mapping)

In [5]:
olympiad_fields = [
    'Принимали ли вы участие в олимпиадах по профильным предметам? / Математика',
    'Принимали ли вы участие в олимпиадах по профильным предметам? / Информатика и программирование',
    'Принимали ли вы участие в олимпиадах по профильным предметам? / Физика'
]

for field in olympiad_fields:
    data[field] = data[field].apply(lambda x: True if pd.notna(x) else False)

In [6]:
hobbies_fields = [
    'Какие увлечения у вас были, когда вы учились в школе? / Геймдев, VR, AR',
    'Какие увлечения у вас были, когда вы учились в школе? / Спорт',
    'Какие увлечения у вас были, когда вы учились в школе? / Программирование',
    'Какие увлечения у вас были, когда вы учились в школе? / Робототехника',
    'Какие увлечения у вас были, когда вы учились в школе? / 3D-моделирование',
    'Какие увлечения у вас были, когда вы учились в школе? / Решение математических задач',
    'Какие увлечения у вас были, когда вы учились в школе? / Творчество: музыка, рисование, танцы и т.п.',
    'Какие увлечения у вас были, когда вы учились в школе? / Физика',
]

for field in hobbies_fields:
    data[field] = data[field].apply(lambda x: True if pd.notna(x) else False)

In [7]:
opinion_mapping = {
    'Да, это точно про меня!': 3,
    'Затрудняюсь ответить': 2,
    'Меня это мало интересует': 1
}

for column in data.columns:
    if any(data[column].isin(opinion_mapping.keys())):
        data[column] = data[column].map(opinion_mapping)


In [8]:
it_field = 'Если бы вы могли выбрать только 1 сферу в IT, то что бы выбрали?'
data = pd.get_dummies(data, columns=[it_field], prefix='', prefix_sep='')

In [9]:
target = data['Направление подготовки']

In [10]:
data['target'] = target

In [11]:
data = data.drop('Направление подготовки', axis=1)

In [12]:
data.columns = data.columns.str.replace('Какие увлечения у вас были, когда вы учились в школе?', '', regex=False)
data.columns = data.columns.str.replace('Принимали ли вы участие в олимпиадах по профильным предметам?', 'ОЛ', regex=False)
data.columns = data.columns.str.replace('Насколько вы любите заниматься ...?', 'НЗ', regex=False)
data.columns

Index(['Пол', 'Сумма баллов ЕГЭ за 3 предмета', 'ОЛ / Математика',
       'ОЛ / Информатика и программирование', 'ОЛ / Физика',
       ' / Геймдев, VR, AR', ' / Спорт', ' / Программирование',
       ' / Робототехника', ' / 3D-моделирование',
       ' / Решение математических задач',
       ' / Творчество: музыка, рисование, танцы и т.п.', ' / Физика',
       'НЗ / Веб-разработкой', 'НЗ / Системным администрированием',
       'НЗ / Математикой',
       'НЗ / Работой с компьютерным оборудованием и сборкой ПК',
       'НЗ / Биологией', 'НЗ / Аналитикой данных',
       'НЗ / Разработка графических интерфейсов - GUI',
       'НЗ / Написанием серверной части приложений - Back',
       'НЗ / Написанием клиентской части приложений - Front',
       'НЗ / Экономикой и финансами',
       'НЗ / Поиском ошибок и уязвимостей на сайтах, в играх или приложениях',
       'Вы поступили в университет, потому что сами этого хотели?',
       'Вам важно получать хорошие отметки?', 'QA-тестировщик',
       '

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 37 columns):
 #   Column                                                                Non-Null Count  Dtype 
---  ------                                                                --------------  ----- 
 0   Пол                                                                   426 non-null    bool  
 1   Сумма баллов ЕГЭ за 3 предмета                                        426 non-null    int64 
 2   ОЛ / Математика                                                       426 non-null    bool  
 3   ОЛ / Информатика и программирование                                   426 non-null    bool  
 4   ОЛ / Физика                                                           426 non-null    bool  
 5    / Геймдев, VR, AR                                                    426 non-null    bool  
 6    / Спорт                                                              426 non-null    bool  
 7    / Прогр

In [14]:
for col in data.select_dtypes(include='int64').columns:
    if col != 'Сумма баллов ЕГЭ за 3 предмета':
        data[col] = data[col].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 37 columns):
 #   Column                                                                Non-Null Count  Dtype   
---  ------                                                                --------------  -----   
 0   Пол                                                                   426 non-null    bool    
 1   Сумма баллов ЕГЭ за 3 предмета                                        426 non-null    int64   
 2   ОЛ / Математика                                                       426 non-null    bool    
 3   ОЛ / Информатика и программирование                                   426 non-null    bool    
 4   ОЛ / Физика                                                           426 non-null    bool    
 5    / Геймдев, VR, AR                                                    426 non-null    bool    
 6    / Спорт                                                              426 non-null    bool

In [15]:
frequency_table = data['Сумма баллов ЕГЭ за 3 предмета'].value_counts().reset_index()
frequency_table.columns = ['Категория', 'Частота']
counts = frequency_table.set_index('Категория')['Частота'].to_dict()

category_params = {
    1: (135, 4),
    2: (175.5, 3.5),
    3: (215.5, 2.5),
    4: (245.5, 2),
    5: (275, 4) 
}

data['Средние баллы'] = data['Сумма баллов ЕГЭ за 3 предмета'].replace({k: v[0] for k, v in category_params.items()})

categorical_samples = data['Средние баллы'].values

def generate_scores_fixed_count(counts):
    generated_scores = []

    for category, count in counts.items():
        mean, std_dev = category_params[category]

        scores = np.random.normal(mean, std_dev, count)

        if category == 1:
            scores = np.clip(scores, 120, 150)
        elif category == 2:
            scores = np.clip(scores, 151, 200)
        elif category == 3:
            scores = np.clip(scores, 201, 230)
        elif category == 4:
            scores = np.clip(scores, 231, 260)
        elif category == 5:
            scores = np.clip(scores, 261, 280)

        generated_scores.extend(scores)

    return np.array(generated_scores)

# Генерация новых баллов
new_generated_scores = generate_scores_fixed_count(counts)

data['Сумма баллов ЕГЭ за 3 предмета'] = new_generated_scores
data = data.drop('Средние баллы', axis=1)

In [16]:
data.to_csv('NSTU_Students_data_transformed.csv')

In [ ]:
ready_data = pd.read_csv('NSTU_Students_data_transformed.csv')
X = data.drop('target', axis=1) 
y = data['target'] 

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
y_array = y.values.reshape(-1, 1)

y_one_hot = encoder.fit_transform(y_array)
y_labels = np.argmax(y_one_hot, axis=1) 


# Модель для предикта

In [93]:
model_all = AdaBoostClassifier(
    estimator=RandomForestClassifier(n_estimators=3, max_depth=2, random_state=42),
    n_estimators=100,
    algorithm='SAMME.R',
    random_state=42
)

model_all.fit(X, y_labels)
probabilities_cv = model_all.predict_proba(X)
pred_cv = model_all.predict(X)

# def calculate_accuracy_top_n(probabilities, y_true, n):
#     top_n_predictions = np.argsort(probabilities, axis=1)[:, -n:]
#     correct_count = 0
#     unique_classes = np.unique(y_true)

#     for i, true_label in enumerate(y_true):
#         predicted_indices = top_n_predictions[i]
#         if true_label in unique_classes[predicted_indices]:
#             correct_count += 1

#     accuracy = correct_count / len(y_true)
#     print(f'Процент правильных ответ для {n} рекомендаций: {accuracy:.2f}')


# calculate_accuracy_top_n(probabilities_cv, y, n=1)
# calculate_accuracy_top_n(probabilities_cv, y, n=2)
# calculate_accuracy_top_n(probabilities_cv, y, n=3)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [96]:
categories = encoder.categories_[0]  
categories

array(['Биотехнические системы и технологии',
       'Информатика и вычислительная техника',
       'Информационная безопасность',
       'Информационная безопасность автоматизированных систем',
       'Информационные системы и технологии', 'Приборостроение',
       'Прикладная информатика', 'Программная инженерия',
       'Управление в технических системах'], dtype=object)

In [94]:
accuracy_score(pred_cv, y_labels)

0.528169014084507

# ONNX MODEL

## Создаем модель в файл

In [110]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert

initial_type = [('float_input', FloatTensorType([None, X.shape[1]]))]

onnx_model = convert_sklearn(model_all, initial_types=initial_type, target_opset=13)

with open("model_with_probabilities.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


## Импортируем модель и проверяем

In [109]:
session = ort.InferenceSession("model_with_softmax.onnx")

# Подготовка входных данных: 2D массив для X (например, 100 примеров с 10 признаками)
inputs = {session.get_inputs()[0].name: np.array(X, dtype=np.float32)}

# Выполнение предсказания
probabilities = session.run(None, inputs)

# Проверка, что возвращается 2D массив
print(probabilities[0].shape)  # Должен быть массив [n_samples, n_classes]

InvalidGraph: [ONNXRuntimeError] : 10 : INVALID_GRAPH : Load model from model_with_softmax.onnx failed:This is an invalid model. Type Error: Type 'tensor(string)' of input parameter (output_label) of operator (Softmax) in node () is invalid.

In [111]:
import onnx
from onnx import helper, TensorProto

# Загружаем модель ONNX
onnx_model = onnx.load("model_with_probabilities.onnx")

# Просмотр типов данных входных и выходных тензоров
for input in onnx_model.graph.input:
    print(f"Input: {input.name}, type: {input.type.tensor_type.elem_type}")

for output in onnx_model.graph.output:
    print(f"Output: {output.name}, type: {output.type.tensor_type.elem_type}")

# Исправляем тип данных для оператора Softmax
softmax_node = onnx_model.graph.node[-1]  # Получаем последний узел (Softmax)

# Изменяем тип данных для выходных тензоров оператора Softmax
for i in range(len(softmax_node.output)):
    # Получаем выходной тензор
    output_name = softmax_node.output[i]
    
    # Находим этот выход в графе
    for value_info in onnx_model.graph.value_info:
        if value_info.name == output_name:
            # Меняем тип данных на float32
            value_info.type.tensor_type.elem_type = TensorProto.FLOAT

# Сохраняем исправленную модель
onnx.save(onnx_model, "model_with_softmax_fixed.onnx")


Input: float_input, type: 1
Output: output_label, type: 7
Output: output_probability, type: 0


In [2]:
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert
from onnx import helper, TensorProto
onnx_model = onnx.load("model.onnx")

onnx.checker.check_model(onnx_model)

ort_session = ort.InferenceSession("model.onnx")

NameError: name 'ort' is not defined

In [113]:
input_data = X.astype(np.float32).values

In [114]:
input_data[0]

array([  1.    , 245.1589,   1.    ,   1.    ,   0.    ,   0.    ,
         1.    ,   1.    ,   0.    ,   0.    ,   0.    ,   0.    ,
         0.    ,   3.    ,   1.    ,   2.    ,   2.    ,   1.    ,
         3.    ,   1.    ,   3.    ,   2.    ,   2.    ,   1.    ,
         1.    ,   1.    ,   0.    ,   0.    ,   0.    ,   1.    ,
         0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ],
      dtype=float32)

In [115]:
ort_inputs = {ort_session.get_inputs()[0].name: input_data}
ort_outs = ort_session.run(None, ort_inputs)

predicted_class = ort_outs[0]

In [116]:
accuracy_score(predicted_class, y)

0.0

In [117]:
type(predicted_class)

numpy.ndarray

In [118]:
predicted_class

array([1, 1, 1, 2, 4, 2, 1, 2, 6, 1, 0, 1, 1, 0, 1, 2, 0, 6, 0, 2, 1, 0,
       1, 6, 7, 1, 1, 1, 1, 6, 3, 6, 7, 1, 1, 1, 2, 2, 6, 6, 7, 1, 1, 0,
       6, 1, 1, 2, 2, 1, 1, 1, 7, 1, 1, 2, 1, 1, 6, 1, 1, 2, 2, 6, 1, 1,
       1, 1, 1, 1, 6, 7, 0, 0, 2, 2, 1, 1, 2, 6, 1, 4, 2, 7, 0, 7, 1, 1,
       6, 6, 4, 7, 1, 7, 0, 6, 1, 6, 7, 1, 7, 7, 1, 1, 6, 6, 1, 2, 6, 4,
       7, 1, 7, 3, 1, 1, 1, 2, 7, 0, 0, 1, 1, 2, 0, 7, 1, 7, 0, 6, 1, 1,
       7, 0, 1, 6, 1, 2, 1, 4, 6, 7, 6, 5, 6, 6, 6, 6, 1, 6, 6, 6, 4, 6,
       6, 1, 7, 7, 4, 1, 4, 7, 1, 0, 6, 2, 6, 1, 2, 1, 6, 1, 1, 1, 2, 1,
       1, 5, 2, 1, 1, 1, 1, 5, 1, 1, 0, 5, 5, 7, 4, 5, 0, 7, 7, 0, 0, 0,
       1, 2, 2, 0, 7, 1, 0, 2, 3, 7, 2, 7, 7, 1, 7, 7, 7, 3, 2, 2, 1, 0,
       2, 2, 1, 3, 2, 7, 3, 3, 1, 1, 1, 7, 7, 1, 8, 8, 8, 8, 2, 8, 8, 3,
       3, 1, 2, 1, 1, 1, 1, 7, 3, 1, 1, 1, 1, 3, 2, 7, 1, 2, 0, 8, 1, 3,
       1, 8, 1, 3, 1, 3, 3, 2, 5, 3, 3, 3, 3, 7, 2, 2, 6, 8, 8, 2, 0, 1,
       1, 1, 6, 1, 6, 2, 3, 3, 8, 6, 0, 3, 1, 7, 2,